<a href="https://colab.research.google.com/github/Abhinav-Jindal05/freecodecamp/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset['sex'] = dataset['sex'].apply(lambda x: x == 'male').astype(int)

In [ ]:
dataset['smoker'] = dataset['smoker'].apply(lambda x: x == 'yes').astype(int)

In [ ]:
dataset = pd.get_dummies(dataset, columns=['region'], drop_first=True)

In [ ]:
dataset['children'].value_counts()

In [ ]:
scaler_age = StandardScaler()
scaler_bmi = StandardScaler()
scaler_children = StandardScaler()
dataset['age'] = scaler_age.fit_transform(dataset['age'].values.reshape(-1,1))
dataset['bmi'] = scaler_bmi.fit_transform(dataset['bmi'].values.reshape(-1,1))
dataset['children'] = scaler_children.fit_transform(dataset['children'].values.reshape(-1,1))

In [ ]:
dataset

In [ ]:
dataset_array = np.array(dataset.loc[:,['age','sex','bmi','children','smoker','region_1','region_2','region_3']],dtype=np.float64)
labels = np.array(dataset.loc[:,'expenses']).flatten()
print(dataset_array.shape)
print(labels.shape)

In [ ]:
print(dataset_array.dtype)

In [ ]:
train_dataset,test_dataset,train_labels,test_labels = train_test_split(dataset_array,labels,test_size=0.2,random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear',
                          kernel_regularizer=regularizers.l2(0.01)
                          )
])
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mse']
)
model.fit(
    train_dataset, train_labels,
          epochs=200,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)],
          verbose=0
    )

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()
# print(test_predictions)

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(test_labels, test_predictions)
print(f"R² Score: {r2:.2f}")
